In [16]:
# https://www.kaggle.com/aashita/nyt-comments/downloads/ArticlesMay2017.csv/notebook


import pandas as pd
import numpy as np
import os,string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,Dropout

In [17]:

all_headlines = []

for filename in os.listdir('Dataset/'):
    if 'Articles' in filename:
        article_df = pd.read_csv("Dataset/" + filename)
        all_headlines.extend(list(article_df.headline.values))
        break;
        
        

In [18]:
all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

831

In [19]:
# Dataset cleaning
# In dataset preparation step, 
# we will first perform text cleaning of the data which includes removal of punctuations and lower casing all the words.

def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

In [20]:
corpus = [clean_text(x) for x in all_headlines]
print(len(corpus))
corpus

831


['finding an expansive view  of a forgotten people in niger',
 'and now  the dreaded trump curse',
 'venezuelas descent into dictatorship',
 'stain permeates basketball blue blood',
 'taking things for granted',
 'the caged beast awakens',
 'an everunfolding story',
 'oreilly thrives as settlements add up',
 'mouse infestation',
 'divide in gop now threatens trump tax plan',
 'variety puzzle acrostic',
 'they can hit a ball 400 feet but play catch thats tricky',
 'in trump country shock at trump budget cuts',
 'why is this hate different from all other hate',
 'pick your favorite ethical offender',
 'my sons growing black pride',
 'jerks and the startups they ruin',
 'trump  needs  a brain',
 'manhood in the age of trump',
 'the value of a black college',
 'initial description',
 'rough estimates',
 'el pasatiempo nacional',
 'cooling off on a hot day at yankee stadium',
 'trumps staff mixed politics and paydays',
 'a virtuoso rebuilding act requires everyone in tune',
 'homeland seaso

In [21]:
# Generating Sequence of N-gram Tokens

tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    
    # tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    print(total_words)
    
     ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words



In [22]:
inp_sequences, total_words = get_sequence_of_tokens(corpus)

2422


In [23]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

In [24]:
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [25]:
print("max_sequence_len = ",max_sequence_len)
print("total_words = ",total_words)

max_sequence_len =  19
total_words =  2422


In [26]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

In [27]:
model = create_model(max_sequence_len, total_words)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 18, 10)            24220     
_________________________________________________________________
lstm (LSTM)                  (None, 100)               44400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 2422)              244622    
Total params: 313,242
Trainable params: 313,242
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.fit(predictors, label, epochs=100, verbose=5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100

In [29]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [35]:
print(generate_text("united states", 5, model, max_sequence_len))
print (generate_text("preident trump", 4, model, max_sequence_len))
print (generate_text("donald trump", 4, model, max_sequence_len))
print (generate_text("india and china", 4, model, max_sequence_len))
print (generate_text("new york", 4, model, max_sequence_len))
print (generate_text("science and technology", 5, model, max_sequence_len))
print (generate_text("india", 5, model, max_sequence_len))
print (generate_text("india", 5, model, max_sequence_len))

United States Plans To Visit White Leads
Preident Trump Is Wimping If Expand
Donald Trump A Psychotic Break At
India And China Fish That Can Even
New York Today A Belated Budget
Science And Technology Text Guy De Maupassants The
India In A Bowl Gov Plan
India In A Bowl Gov Plan
